<a href="https://colab.research.google.com/github/suleman12344/Final_YearProject/blob/main/Model/finalYear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
from matplotlib import pyplot as plt

In [2]:
DATASET_PATH = '/content/drive/MyDrive/cubicasa5k'
TRAIN_PATH = '/content/drive/MyDrive/cubicasa5k/train.txt'
VAL_PATH = '/content/drive/MyDrive/cubicasa5k/val.txt'
TEST_PATH = '/content/drive/MyDrive/cubicasa5k/test.txt'

In [47]:
# Dataset with annotations (dummy user info for conditioning)
class FloorplanDataset(Dataset):
    def __init__(self, txt_file, base_dir, transform=None):
        self.base_dir = base_dir
        self.transform = transform

        # Read file and prepare image paths
        with open(txt_file, 'r') as file:
            folders = [line.strip() for line in file.readlines()]


        self.image_files = []
        for folder in folders:
            folder_path = base_dir+folder #os.path.join(base_dir, folder)
            self.image_files += [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

        # Dummy annotations (e.g., number of rooms, area)
        self.annotations = torch.randn(len(self.image_files), 2)  # Replace with real data

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        annotation = self.annotations[idx]  # Conditioning info
        return image, annotation

# Transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
])


In [48]:
# Datasets and DataLoaders
train_dataset = FloorplanDataset(txt_file=TRAIN_PATH, base_dir=DATASET_PATH, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [11]:
class Generator(nn.Module):
    def __init__(self, latent_dim, condition_dim):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(latent_dim + condition_dim, 1024)
        self.fc2 = nn.Linear(1024, 256 * 16 * 16)
        self.main = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, noise, condition):
        x = torch.cat((noise, condition), dim=1)
        x = self.fc1(x)
        x = self.fc2(x).view(-1, 256, 16, 16)
        return self.main(x)

In [41]:
class Discriminator(nn.Module):
    def __init__(self, condition_dim):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1024, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True)
           )
        self.linear = nn.Sequential(
            nn.Flatten(),
            nn.Linear(0,1), #placeholder, updated in forward()
        )
        # Use condition_dim to create a layer for condition embedding
        self.condition_embedding = nn.Linear(condition_dim, 1024)  # Example embedding size
        self.final_linear = nn.Sequential(
            nn.Linear(0,1), #placeholder, updated in forward()
            nn.Sigmoid()
        )

    def forward(self, x, conditions): # Modified to accept conditions
        x = self.main(x)

        # Calculate the size dynamically based on the input
        linear_input_size = x.view(x.size(0), -1).shape[1]
        self.linear[1] = nn.Linear(linear_input_size, 1024)  # update the linear layer
        x = self.linear(x)

        embedded_conditions = self.condition_embedding(conditions)

        # Concatenate after flattening
        x = torch.cat([x, embedded_conditions], dim=1)

        self.final_linear[0] = nn.Linear(x.shape[1], 1)
        x = self.final_linear(x)

        return x # Return the combined output

In [42]:
# Initialize models
latent_dim = 100
condition_dim = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = Generator(latent_dim, condition_dim).to(device)
discriminator = Discriminator(condition_dim).to(device)

# Loss and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [46]:
# Training loop
epochs = 5
for epoch in range(epochs):  # This loop should increment epoch correctly
    print(f"Epoch {epoch+1}/{epochs}")
    for i, (real_images, conditions) in enumerate(train_loader):
        print(f"Batch {i+1}/{len(train_loader)}")
        batch_size = real_images.size(0)
        real_images, conditions = real_images.to(device), conditions.to(device)

        # Train Discriminator
        optimizer_D.zero_grad()
        real_labels = torch.ones(batch_size, 1, device=device)
        fake_labels = torch.zeros(batch_size, 1, device=device)

        real_preds = discriminator(real_images, conditions)
        real_loss = criterion(real_preds, real_labels)

        noise = torch.randn(batch_size, latent_dim, device=device)
        fake_images = generator(noise, conditions)
        fake_preds = discriminator(fake_images.detach(), conditions)
        fake_loss = criterion(fake_preds, fake_labels)

        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        fake_preds = discriminator(fake_images, conditions)
        g_loss = criterion(fake_preds, real_labels)
        g_loss.backward()
        optimizer_G.step()

        # Logging
        if i % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")


Epoch 1/5
Batch 1/162
Epoch [1/5], Step [1/162], D Loss: 1.3356, G Loss: 0.5954
Batch 2/162
Batch 3/162
Batch 4/162
Batch 5/162
Batch 6/162
Batch 7/162
Batch 8/162
Batch 9/162
Batch 10/162


KeyboardInterrupt: 

In [ ]:
  # Save models
os.makedirs("generated_images", exist_ok=True)
noise = torch.randn(16, latent_dim, device=device)
conditions = torch.randn(16, condition_dim, device=device)
fake_images = generator(noise, conditions)
save_image(fake_images, "generated_images/sample.png", nrow=4, normalize=True)
torch.save(generator.state_dict(), "generator.pth")
torch.save(discriminator.state_dict(), "discriminator.pth")
print("Training complete. Models saved.")

In [44]:
print(f"Number of samples in dataset: {len(train_dataset)}")
print(f"Number of batches per epoch: {len(train_loader)}")

Number of samples in dataset: 10336
Number of batches per epoch: 162
